In [25]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)

In [26]:
df = pd.read_csv("persona.csv")
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [27]:
df.shape

(5000, 5)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [29]:
df.describe()

,PRICE,AGE
count,5000.000000,5000.000000
mean,34.132000,23.581400
std,12.464897,8.995908
min,9.000000,15.000000
25%,29.000000,17.000000
50%,39.000000,21.000000
75%,39.000000,27.000000
max,59.000000,66.000000


In [30]:
df["SOURCE"].nunique()

2

In [31]:
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [32]:
df["PRICE"].nunique()

6

In [33]:
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

In [34]:
df["COUNTRY"].value_counts()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

In [35]:
df.groupby("COUNTRY")["PRICE"].sum() #.agg({"PRICE": "sum"})

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

In [36]:
df.groupby("SOURCE")["PRICE"].count() #.agg({"PRICE":"count"})

SOURCE
android    2974
ios        2026
Name: PRICE, dtype: int64

In [37]:
df.groupby("COUNTRY").agg({"PRICE":"mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [38]:
df.groupby("SOURCE").agg({"PRICE":"mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [39]:
df.groupby(["COUNTRY","SOURCE"]).agg({"PRICE":"mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [40]:
agg_df = df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"PRICE":"mean"}).sort_values(by = "PRICE" , ascending=False)
agg_df

PRICE
COUNTRY SOURCE  SEX    AGE       
bra     android male   46    59.0
usa     android male   36    59.0
fra     android female 24    59.0
usa     ios     male   32    54.0
deu     android female 36    49.0
...                           ...
usa     ios     female 38    19.0
                       30    19.0
can     android female 27    19.0
fra     android male   18    19.0
deu     android male   26     9.0

[348 rows x 1 columns]

In [41]:
agg_df.reset_index(inplace=True)
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0
...,...,...,...,...,...
343,usa,ios,female,38,19.0
344,usa,ios,female,30,19.0
345,can,android,female,27,19.0
346,fra,android,male,18,19.0


In [42]:
df["AGE"].describe()

count    5000.000000
mean       23.581400
std         8.995908
min        15.000000
25%        17.000000
50%        21.000000
75%        27.000000
max        66.000000
Name: AGE, dtype: float64

In [60]:
bins = [0,18,23,30,40,agg_df["AGE"].max()]
labels = ["0_18","19_23","24_30","31_40","41_" + str(agg_df["AGE"].max())]

agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins= bins, labels = labels)

In [44]:
# agg_df["customers_level_based"] = agg_df[["COUNTRY" , "SOURCE" , "SEX" , "AGE_CAT"]].agg(lambda x: '_'.join(x).upper(), axis=1)

In [61]:
agg_df["customers_level_based"] = (agg_df["COUNTRY"].astype(str) + "_" +
                              agg_df["SOURCE"].astype(str) + "_" +
                              agg_df["SEX"].astype(str) + "_"+
                              agg_df["AGE_CAT"].astype(str)).str.upper()
agg_df["customers_level_based"]

0        BRA_ANDROID_MALE_41_66
1        USA_ANDROID_MALE_31_40
2      FRA_ANDROID_FEMALE_24_30
3            USA_IOS_MALE_31_40
4      DEU_ANDROID_FEMALE_31_40
                 ...           
343        USA_IOS_FEMALE_31_40
344        USA_IOS_FEMALE_24_30
345    CAN_ANDROID_FEMALE_24_30
346       FRA_ANDROID_MALE_0_18
347      DEU_ANDROID_MALE_24_30
Name: customers_level_based, Length: 348, dtype: object

In [62]:
agg_df["Segment"] = pd.qcut(agg_df["PRICE"], 4 , labels = ["D","C","B","A"])
agg_df["Segment"]

0      A
1      A
2      A
3      A
4      A
      ..
343    D
344    D
345    D
346    D
347    D
Name: Segment, Length: 348, dtype: category
Categories (4, object): ['D' < 'C' < 'B' < 'A']

In [63]:
agg_df.groupby("Segment").agg({"PRICE": ["mean","max","sum"]})

PRICE                        
              mean        max          sum
Segment                                   
D        27.302596  31.105263  2375.325850
C        32.933339  34.000000  3128.667165
B        35.436170  37.000000  2870.329792
A        41.434736  59.000000  3521.952577

In [65]:
new_customer = "TUR_ANDROID_FEMALE_31_40"

agg_df[agg_df["customers_level_based"] == new_customer]

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based,Segment
18,tur,android,female,32,43.000000,31_40,TUR_ANDROID_FEMALE_31_40,A
35,tur,android,female,31,40.666667,31_40,TUR_ANDROID_FEMALE_31_40,A
